In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyarrow
from dataclasses import dataclass
import yaml
from typing import Union

In [2]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [3]:
os.chdir("..")
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [4]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Класс `DropBatchProcessor`

In [5]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

**Тест `DropBatchProcessor`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.processor import DropBatchHandler
from data_generator.general_time import pd_timestamp_to_unix

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
# configs = drop_cfg_build.build_dist_cfg()
configs = drop_cfg_build.build_purch_cfg()

# base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
# base_agg1.build_all(drop_type="purchaser")
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

batch_hand = DropBatchHandler(base=base_agg1, create_txn=cr_drop_txn1)

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=2337, district_id=12, birth_date='1917-04-09', sex='female', region='Карелия', area='Петрозаводск', timezone='UTC+3', lat=61.78909, lon=34.3596263, population=263540, home_ip='2.60.8.164', geometry=<MULTIPOLYGON (((34.186 61.851, 34.191 61.851, 34.195 61.851, 34.197 61.851,...>)

In [3]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data, batch_hand):
    cr_drop_txn.reset_cache(only_counters=False)
    behav_hand.reset_cache(all=True)
    amt_hand.reset_cache(all=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()
    batch_hand.reset_cache(all=True)

In [4]:
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))].sample(1)
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))]
# acc_hand1.reset_cache()
acc_hand1.used_accounts

Series([], Name: account_id, dtype: object)

**`should_decline`**  
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1

In [5]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1
batch_hand.should_decline()

True

### **Тест метода `distributor`**

**баланс > trf_max** - выводить кол-во попыток через print внутри atts_after_decline()  
4 кейса
- scen=`"split_transfer"`, in_chunks = `True`
- scen=`"atm+transfer`", in_chunks = `True`
- scen=`"atm"`, in_chunks = `False`
- scen=`"transfer"`, in_chunks = `False`

In [10]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [11]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.trf_max + 1000
    behav_hand1.scen = "transfer"
    behav_hand1.in_chunks = False

    batch_hand.distributor()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

attempts 3


In [4]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df

**С генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []

while True:
    declined = batch_hand.declined
    receive_txn1 = cr_drop_txn1.trf_or_atm(dist=True, receive=True, to_drop=False, declined=declined)
    all_txns1.append(receive_txn1)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

In [9]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df #.query("type != 'inbound'").shape

16


(11, 19)

### **Тест метода `purchaser`**

**баланс > amt_max** - выводить кол-во попыток через print внутри atts_after_decline()
- scen=`"split_money"`, in_chunks = `True`
- scen=`"transfer"`, in_chunks = `False`

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns2 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [8]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.amt_max + 1000
    behav_hand1.scen = "one_purchase"
    behav_hand1.in_chunks = False

    batch_hand.purchaser()
    txns_fm_batch2 = batch_hand.txns_fm_batch
    all_txns2.extend(txns_fm_batch2)
    batch_hand.reset_cache(all=False)

attempts 0


In [10]:
all_txns_df2 = pd.DataFrame(all_txns2)
print(all_txns_df2.shape[0])
all_txns_df2

**`purchaser` с генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [5]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns3 = []

while True:
    declined = batch_hand.declined
    receive_txn3 = cr_drop_txn1.trf_or_atm(dist=False, receive=True, to_drop=False, declined=declined)
    all_txns3.append(receive_txn3)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch3 = batch_hand.txns_fm_batch
    all_txns3.extend(txns_fm_batch3)
    batch_hand.reset_cache(all=False)

attempts 3


In [7]:
all_txns_df3 = pd.DataFrame(all_txns3)
print(all_txns_df3.shape[0])
all_txns_df3 #.query("type != 'inbound'").shape

17


(12, 19)

## Класс `DropLifecycleManager`
- Полный жизненный цикл дропа

In [ ]:
# ----------ВСТАВИТЬ ГОТОВЫЙ КЛАСС-------------------------

**Тест `DropLifecycleManager`**

In [ ]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [ ]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
# from data_generator.fraud.drops.processor import DropBatchHandler
# from data_generator.general_time import pd_timestamp_to_unix
from data_generator.fraud.drops.simulator import DropLifecycleManager

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_dist_cfg()
# configs = drop_cfg_build.build_purch_cfg()

base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
# base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

life_manager = DropLifecycleManager(base=base_agg1, create_txn=cr_drop_txn1)
batch_hand = DropBatchHandler(base=base_agg1, create_txn=cr_drop_txn1)

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info